In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
json_list = list()
with open('tweets_DM.json' , 'r') as file:
    for line in file:
        json_list.append(json.loads(line))

In [3]:
tweet_list = list()
for json in json_list:
    tweet_id = json['_source']['tweet']['tweet_id']
    text = json['_source']['tweet']['text']
    tweet_list.append([tweet_id, text])

In [4]:
# tweet_list

In [5]:
text_df = pd.DataFrame(tweet_list, columns=['tweet_id', 'text'])

In [6]:
emotion_df = pd.read_csv('emotion.csv')
# emotion_df

In [7]:
identification_df = pd.read_csv('data_identification.csv')
# identification_df

In [8]:
test_df = identification_df[identification_df['identification'] == 'test']
# test_df

In [9]:
train_df = text_df.merge(emotion_df, left_on='tweet_id', right_on='tweet_id')
# train_df

In [10]:
x_list = train_df['text'].to_list()
y_list = train_df['emotion'].to_list()

In [11]:
# x_list = list()
# for i in x:
#     x_list.append([i])

In [12]:
# y_list = list()
# for i in y:
#     y_list.append([i])

In [13]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x_list, y_list, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [14]:
print(len(train_x), len(train_y))
print(len(test_x), len(test_y))
print(len(valid_y), len(valid_y))

931560 931560
291113 291113
232890 232890


In [15]:
import kashgari
from kashgari.tasks.classification import BiLSTM_Model
kashgari.config.use_cudnn_cell = True
import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()
model.fit(train_x, 
          train_y, 
          valid_x, 
          valid_y,
          epochs=100,
         )
model.save('model/BiLSTM_100_Model')

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 137)]             0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 137, 100)          239000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 8)                 2056      
Total params: 475,552
Trainable params: 475,552
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
14556/14556 [==============================] - 1953s 134ms/step - loss: 1.5012 - acc: 0.4529 - val_loss: 1.3108 - val_acc: 0.5235
Epoch 2/100
14556/14556 [==============================] - 1908s 131ms/step - loss: 1.2528 - acc: 0.5453 - va

14556/14556 [==============================] - 2119s 146ms/step - loss: 0.9954 - acc: 0.6397 - val_loss: 1.1331 - val_acc: 0.5948
Epoch 53/100
14556/14556 [==============================] - 2114s 145ms/step - loss: 0.9965 - acc: 0.6394 - val_loss: 1.1334 - val_acc: 0.5936
Epoch 54/100
14556/14556 [==============================] - 2131s 146ms/step - loss: 0.9949 - acc: 0.6400 - val_loss: 1.1328 - val_acc: 0.5965
Epoch 55/100
14556/14556 [==============================] - 2120s 146ms/step - loss: 0.9967 - acc: 0.6394 - val_loss: 1.1362 - val_acc: 0.5960
Epoch 56/100
14556/14556 [==============================] - 2115s 145ms/step - loss: 0.9993 - acc: 0.6384 - val_loss: 1.1315 - val_acc: 0.5973
Epoch 57/100
14556/14556 [==============================] - 2120s 146ms/step - loss: 0.9979 - acc: 0.6387 - val_loss: 1.1312 - val_acc: 0.5971
Epoch 58/100
14556/14556 [==============================] - 2068s 142ms/step - loss: 0.9960 - acc: 0.6397 - val_loss: 1.1330 - val_acc: 0.5951
Epoch 59/100

In [16]:
model = kashgari.utils.load_model('model/BiLSTM_100_Model')
model.evaluate(test_x, test_y)

              precision    recall  f1-score   support

       anger     0.5805    0.2645    0.3634      7946
anticipation     0.6786    0.6058    0.6401     49984
     disgust     0.4643    0.4330    0.4481     27669
        fear     0.7140    0.4301    0.5368     12846
         joy     0.6137    0.7887    0.6903    102943
     sadness     0.4911    0.5800    0.5319     38745
    surprise     0.7654    0.2154    0.3361      9816
       trust     0.5874    0.4069    0.4808     41164

    accuracy                         0.5923    291113
   macro avg     0.6119    0.4656    0.5035    291113
weighted avg     0.5993    0.5923    0.5803    291113



In [17]:
# import kashgari
# from kashgari.tasks.classification import BiLSTM_Model
# from kashgari.embeddings import BERTEmbedding
# kashgari.config.use_cudnn_cell = True
# import logging
# logging.basicConfig(level='DEBUG')

# bert_embed = BERTEmbedding('embedding/bert_base/',
#                            task=kashgari.CLASSIFICATION,
#                            sequence_length=100,
#                           )

# model = BiLSTM_Model(bert_embed)
# model.fit(train_x, 
#           train_y, 
#           valid_x, 
#           valid_y,
#           epochs=5,
#          )

# model.save('model/BERT_base_BiLSTM_Model')

In [18]:
# model = kashgari.utils.load_model('model/BERT_base_BiLSTM_Model')
# model.evaluate(test_x, test_y)

In [19]:
# train_df

In [20]:
# test_df

In [21]:
test_df = test_df.merge(text_df, left_on='tweet_id', right_on='tweet_id')
# test_df

In [22]:
text_list = test_df['text'].tolist()
# text_list

In [23]:
predict_list = model.predict(text_list)
predict_list

['joy',
 'sadness',
 'joy',
 'joy',
 'disgust',
 'joy',
 'trust',
 'anticipation',
 'anticipation',
 'joy',
 'joy',
 'sadness',
 'sadness',
 'trust',
 'joy',
 'joy',
 'sadness',
 'joy',
 'trust',
 'joy',
 'sadness',
 'joy',
 'anger',
 'joy',
 'sadness',
 'anticipation',
 'disgust',
 'sadness',
 'disgust',
 'sadness',
 'sadness',
 'joy',
 'anticipation',
 'joy',
 'sadness',
 'joy',
 'anticipation',
 'anticipation',
 'sadness',
 'sadness',
 'sadness',
 'sadness',
 'joy',
 'joy',
 'sadness',
 'sadness',
 'anticipation',
 'anticipation',
 'joy',
 'joy',
 'disgust',
 'joy',
 'joy',
 'sadness',
 'joy',
 'disgust',
 'joy',
 'sadness',
 'joy',
 'joy',
 'joy',
 'trust',
 'trust',
 'joy',
 'joy',
 'trust',
 'disgust',
 'sadness',
 'joy',
 'trust',
 'joy',
 'disgust',
 'joy',
 'joy',
 'sadness',
 'joy',
 'anticipation',
 'sadness',
 'joy',
 'disgust',
 'joy',
 'sadness',
 'sadness',
 'trust',
 'sadness',
 'disgust',
 'joy',
 'joy',
 'joy',
 'trust',
 'sadness',
 'anticipation',
 'joy',
 'trust',


In [24]:
test_df['predict'] = predict_list
# test_df

In [25]:
output_df = test_df[['tweet_id', 'predict']]
output_df = output_df.rename(columns={'tweet_id':'id', 'predict':'emotion'})
# output_df

In [26]:
output_df.to_csv('../output/BiLSTM_100_Model.csv', index=False, header=True)